In [4]:
import numpy as np
import pandas as pd
import joblib
import os
import requests
import json
import time
import concurrent.futures
import glob
from tqdm import tqdm

In [5]:
details = glob.glob("details/*.pkl")

In [6]:
data_list = []

In [7]:
for i, pkl in tqdm(enumerate(details), total=len(details)):
    data = joblib.load(pkl)
    data_list.append(data)

100%|██████████| 453/453 [00:02<00:00, 210.61it/s]


In [14]:
df = pd.DataFrame.from_dict(data_list, orient='columns')

In [15]:
df.columns.to_list()

['id',
 'sku',
 'type',
 'sex',
 'color',
 'breed',
 'feed',
 'age_in_month',
 'slides',
 'sale_offer_percentage',
 'feedlot_in_month',
 'teeth',
 'parts_available',
 'height_in_inch',
 'youtube_slug',
 'weight_in_kg',
 'price',
 'is_special',
 'is_dewormed',
 'is_fmd_vaccinated',
 'is_anthrax_vaccinated',
 'is_lumpy_skin_disease',
 'size',
 'thumbnail',
 'created_at',
 'status']

In [16]:
columns = ['sku', 'type', 'sex', 'color', 'breed', 'feed', 'age_in_month', 'feedlot_in_month', 'teeth', 'height_in_inch', 'youtube_slug', 'weight_in_kg', 'price', 'size']
df = df.loc[:, columns]

In [17]:
df.loc[0,:]

sku                                                           BLF2002
type                                                              COW
sex                                                         MALE_BULL
color                                                         NON_RED
breed                                                           LOCAL
feed                [JUMBOO, LUCERNE, NAPIER, SILAGE, STRAW, CORN,...
age_in_month                                                2.5 Years
feedlot_in_month                                             3 Months
teeth                                                               4
height_in_inch                                                   51.0
youtube_slug                                        I0FFZuejRlo?rel=0
weight_in_kg                                                    296.0
price                                                       125000.00
size                                                            LARGE
Name: 0, dtype: obje